In [1]:
import torchao
import torch


/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-23:03:28:28,196 INFO     [utils.py:148] Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-06-23:03:28:28,197 INFO     [utils.py:161] NumExpr defaulting to 8 threads.
2024-06-23:03:28:28,864 INFO     [config.py:58] PyTorch version 2.3.0 available.
2024-06-23:03:28:28,865 INFO     [config.py:95] TensorFlow version 2.16.1 available.
/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled withou

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
from distutils.sysconfig import get_python_lib
prefix = get_python_lib()

In [3]:
import sys; print(sys.path)

torch.ops.load_library(f"{prefix}/torchao-0.3.0-py3.11-macosx-11.1-arm64.egg/torchao/libexecutorch_kernels.dylib")

['/Users/larryliu/ao/torchao/csrc/executorch', '/Users/larryliu/miniconda3/envs/executorch/lib/python311.zip', '/Users/larryliu/miniconda3/envs/executorch/lib/python3.11', '/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/lib-dynload', '', '/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/site-packages', '__editable__.llava-1.2.2.post1.finder.__path_hook__', '/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/site-packages/torchao-0.3.0-py3.11-macosx-11.1-arm64.egg', '/Users/larryliu/CLionProjects/pytorch', '/Users/larryliu/ao', '/var/folders/21/pcyct_g904x1pf8l_b2pvpy00000gn/T/tmp2gzr6njx', '/Users/larryliu/.cache/huggingface/modules']


In [4]:
from executorch.extension.pybindings import portable_lib

In [5]:
print("\n".join(portable_lib._get_operator_names()))

aten::sym_size.int
aten::_local_scalar_dense
aten::sym_numel
executorch_prim::add.Scalar
executorch_prim::sub.Scalar
executorch_prim::mul.Scalar
executorch_prim::floordiv.Scalar
executorch_prim::truediv.Scalar
executorch_prim::sym_float.Scalar
executorch_prim::eq.Scalar
executorch_prim::gt.Scalar
executorch_prim::lt.Scalar
executorch_prim::ge.Scalar
executorch_prim::le.Scalar
executorch_prim::floordiv.int
executorch_prim::et_copy_index.tensor
executorch_prim::et_view.default
aten::_cdist_forward.out
aten::_log_softmax.out
aten::_native_batch_norm_legit.out
aten::_native_batch_norm_legit.no_stats_out
aten::_native_batch_norm_legit_no_training.out
aten::_pdist_forward.out
aten::_softmax.out
aten::_to_copy.out
aten::abs.out
aten::acos.out
aten::acosh.out
aten::add.out
aten::add.Scalar_out
aten::addmm.out
aten::alias_copy.out
aten::amax.out
aten::amin.out
aten::any.all_out
aten::any.dims_out
aten::any.out
aten::arange.out
aten::arange.start_out
aten::argmax.out
aten::argmin.out
aten::as_st

In [6]:
class Demo(torch.nn.Module):
    def forward(self, A, B, scalesAndZeros):
        return torch.ops.torchao.int4mv.default(A, B, 32, scalesAndZeros)

In [7]:
M = 1
K = 4096
N = 4096
A = torch.randn(M, K, device="mps", dtype=torch.float16) # M x K
B = torch.randn(N, K // 2, device="mps", dtype=torch.int8) # N x K / 2
scalesAndZeros = torch.randn(N, K // 32, 2, device="mps", dtype=torch.float16) # N x K / 32 x 2

In [8]:
module = Demo()
module.forward(A, B, scalesAndZeros)

tensor([[-555.5000,  268.7500,  210.6250,  ..., 1293.0000,  815.5000,
          352.7500]], device='mps:0', dtype=torch.float16)

In [9]:
from executorch.exir import EdgeProgramManager, ExecutorchProgramManager, to_edge, EdgeCompileConfig
ep = torch.export.export(module, (A, B, scalesAndZeros))

In [10]:
executorch_program = to_edge(ep, compile_config=EdgeCompileConfig(_check_ir_validity=False)).to_executorch()

In [ ]:
from executorch.extension.pybindings.portable_lib import _load_for_executorch_from_buffer
m = _load_for_executorch_from_buffer(executorch_program.buffer)
out = m.forward([A.to(device="cpu"), B.to(device="cpu"), scalesAndZeros.to(device="cpu")])

In [ ]:
print(executorch_program.exported_program())

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, a: "f16[1, 4096]", b: "i8[4096, 2048]", scalesandzeros: "f16[4096, 128, 2]"):
            # No stacktrace found for following nodes
            alloc: "f16[1, 4096]" = executorch_exir_memory_alloc(((1, 4096), torch.float16))
            
            # File: /var/folders/21/pcyct_g904x1pf8l_b2pvpy00000gn/T/ipykernel_1029/39005463.py:3 in forward, code: return torch.ops.torchao.int4mv.default(A, B, 32, scalesAndZeros)
            torchao_int4mv_default: "f16[1, 4096]" = torch.ops.torchao.int4mv.out(a, b, 32, scalesandzeros, out = alloc);  a = b = scalesandzeros = alloc = None
            return (torchao_int4mv_default,)
            
Graph signature: ExportGraphSignature(input_specs=[InputSpec(kind=<InputKind.USER_INPUT: 1>, arg=TensorArgument(name='a'), target=None, persistent=None), InputSpec(kind=<InputKind.USER_INPUT: 1>, arg=TensorArgument(name='b'), target=None, persistent=None), InputSpec(kind=<Input

In [ ]:
print(out)

[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16)]
